In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#input_size = dimensionality of the image
#hidden_size = size of the network
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size, num_classes):
    super(NeuralNet,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  #forward function : describe the flow of input data through the network layers
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out
  
class OldNet(nn.Module):
    def __init__(self,input_size,hidden_size, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, hidden_size, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(hidden_size, input_size, 5)
        self.fc1 = nn.Linear(input_size * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class Net(nn.Module):
    def __init__(self, in_channels,hidden_size, num_classes, pool_size = 2, kernel_size = 5):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=hidden_size, kernel_size=kernel_size)
        self.pool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.conv2 = nn.Conv2d(in_channels=hidden_size, out_channels=in_channels, kernel_size=kernel_size)

        self.fc1 = nn.Linear(in_features=in_channels * 5 * 5, out_features=hidden_size)
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

torch.cuda.is_available()


True

In [2]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

class datasetMNIST(Dataset):
  def __init__(self,file_path,transform=None) -> None:
    self.data = pd.read_csv(file_path)
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self,index):
    image = self.data.iloc[index,1:].values.astype(np.uint8).reshape((1,28,28))
    label = self.data.iloc[index,0]

    if self.transform is not None:
      image = self.transform(image)

    return image , label


In [3]:
import torchvision
batch_train_size = 4

transformation = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.1307,), (0.3081,))
        ])

CIFAR10trainset = torchvision.datasets.CIFAR10(
        root='./data', 
        train=True,
        download=True, 
        transform=transformation
    )

CIFAR10testnset = torchvision.datasets.CIFAR10(
        root='./data', 
        train=False,
        download=True, 
        transform=transformation
    )

MNISTtrainset = torchvision.datasets.MNIST(
        root='./data',
        train=True,
        transform=transformation,
        download=True
    )

train_loader = torch.utils.data.DataLoader(
    dataset=CIFAR10trainset,
    batch_size=batch_train_size,
    shuffle=True
        )
test_loader = torch.utils.data.DataLoader(
    dataset=CIFAR10testnset,
    batch_size=batch_train_size,
    shuffle=False
        )

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import scipy.io

class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, mat_file, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.jpg')]
        self.data = []

        labels = scipy.io.loadmat(mat_file)['labels'][0]

        for image_path , label in zip(self.image_files , labels):
            image = Image.open(image_path).convert('L')  # 'L' mode for grayscale images
            self.data.append((image, torch.tensor(label)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, label = self.data[index]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

# Esempio di utilizzo:
transformation = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                (0.1307,), (0.3081,))
        ])
dataset = ImageFolderDataset("./data/102flowers/jpg/",mat_file="./data/102flowers/imagelabels.mat", transform=transform)


In [8]:
input_size , hidden_size , num_classes = 32 , 500 , 10
model = Net(input_size,hidden_size,num_classes)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
for epoch in range(10): # loop over the dataset multiple times
    tot_loss, tot_samples = 0.0, 0
    for i, data in enumerate(dataset , 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].cuda(), data[1].cuda()
        tot_samples += inputs.size(0)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        tot_loss += (loss.item() * inputs.size(0))
    print("Epoch %d loss is: %.6f" % (epoch, (tot_loss * 1.0 / float(tot_samples))))


RuntimeError: Given groups=1, weight of size [500, 3, 5, 5], expected input[1, 1, 500, 591] to have 3 channels, but got 1 channels instead

In [60]:
correct = 0
total = 0
model = model.cuda()
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].cuda(), data[1].cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 67 %
